<a href="https://colab.research.google.com/github/CurtCalledBurt/DS-Unit-2-Regression-Classification/blob/master/module3/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 4.3MB/s 
     |████████████████████████████████| 112kB 15.6MB/s 
     |████████████████████████████████| 133kB 59.3MB/s 
     |████████████████████████████████| 57.7MB 1.3MB/s 
     |████████████████████████████████| 235kB 40.4MB/s 
     |████████████████████████████████| 614kB 51.5MB/s 
     |████████████████████████████████| 235kB 47.8MB/s 
     |████████████████████████████████| 307kB 54.8MB/s 
     |████████████████████████████████| 51kB 23.2MB/s 
     |████████████████████████████████| 215kB 55.1MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) (standardize the data).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [6]:
df.shape

(23040, 21)

In [0]:
# checking if SALE_DATE is in datetime yet, if it is not we'll convert it.
df.dtypes;

In [0]:
# It wasn't in datetime, so we'll convert SALE_DATE to datetime
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df.dtypes;

In [0]:
# getting 'one family dwellings' betewen $100,000 and $2,000,000 dollars

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df[ (df['SALE_PRICE'] > 100_000) & (df['SALE_PRICE'] < 2_000_000)]

In [10]:
#checking for columns with many NaNs
print(df.shape)
df.isna().sum();

(3151, 21)


In [0]:
# we see that both EASE-MENT and APARTMENT_NUMBER are either empty or basically empty,
# so we're just going to drop those two columns

df = df[ df.columns.drop(['EASE-MENT', 'APARTMENT_NUMBER']) ]
df.isna().sum();

In [0]:
# Now we scale the data before the split
from sklearn import preprocessing

num_cols = df.select_dtypes(include='number').columns

df[num_cols] = preprocessing.scale(df[num_cols])

In [13]:
#train-test split by month (hence the datetime thing we did earlier)

last_date = pd.to_datetime('2019-04-01')

train = df[df['SALE_DATE'] < last_date] # .dt.month.isin(['01', '02', '03']) ]
test = df[df['SALE_DATE'] >= last_date]  # .dt.month.isin(['04']) ]
train.shape, test.shape

((2507, 19), (644, 19))

In [14]:
# We check the number of unique elements in the object columns to decide which
# columns we are going to one-hot encode.
# probably all of them except the last three: SALE_DATE, LAND_SQUARE_FEET, and ADDRESS
# and the first one, because it doesn't change: BUILDING_CLASS_CATEGORY
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaN,NaN
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaN,NaN
BOROUGH,2507,5,4,1204,NaN,NaN
NEIGHBORHOOD,2507,7,OTHER,2360,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaN,NaN
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaN,NaN
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-03-30 00:00:00
LAND_SQUARE_FEET,2507,887,"4,000",234,NaN,NaN
ADDRESS,2507,2497,22-40 93RD STREET,2,NaN,NaN


In [15]:
# I'm heavily borrowing from what was being done in lecture here to make the feature list
# by removing what I don't want in the feature column as opposed to making it by typing each
# feature that I want. That was always a pain to do.

target = 'SALE_PRICE'

numerics = train.select_dtypes(include='number').columns.drop([target]).tolist()
categoricals = train.select_dtypes(exclude='number').columns.tolist()

low_cardinality = [col for col in categoricals if train[col].nunique() <= 50]

# do_not_one_hot_encode = ['SALE_DATE', 'LAND_SQUARE_FEET', 'ADDRESS', 'BUILDING_CLASS_CATEGORY']

# remember target is not a list, so put brackets around it to make it one so '+' becomes
# concatenation 
# features = train.columns.drop( [target] + do_not_one_hot_encode)

features = numerics + low_cardinality

x_train = train[features]
x_test = test[features]

y_train = train[target]
y_test = test[target]

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2507, 15), (644, 15), (2507,), (644,))

In [16]:
import numpy as np

#We one-hot encode the categorical data
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names = True)

# ALWAYS .fit_transform on TRAIN data set
# ALWAYS .transform on TEST data set
# AND lastly,
# ALWAYS encode both x and y data sets, 
# otherwise you get division by 0 errors later
# Or you'll get division by 0 errors both ways!
x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)
# y_train = encoder.fit_transform(y_train)
# y_test = encoder.transform(y_test)

x_train.shape, x_test.shape, y_train.shape, y_test.to_numpy().shape

((2507, 48), (644, 48), (2507,), (644,))

In [0]:
# sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered
# scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered


#this silences a few divide by 0 warnings we get

# warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='sklearn')
# warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='scipy')

In [18]:
# We select the best features for modeling
from sklearn.feature_selection import f_regression, SelectKBest
selector = SelectKBest(score_func=f_regression, k = 20)

# ALWAYS fit_transform on TRAIN data set
# ALWAYS transform on TEST data set
# Note fit_transform takes the x and y set, transform only takes the x
x_train_selection = selector.fit_transform(x_train, y_train.values.ravel());
x_test_selection = selector.transform(x_test);

x_train_selection.shape, x_test_selection.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equa

((2507, 20), (644, 20))

In [19]:
testing = np.array([1,2,3,4,5,6,7])

y_test.values.ravel().shape, testing.shape

((644,), (7,))

In [20]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

ridge_reg = Ridge().fit(x_train_selection, y_train)

mse = mean_squared_error(y_train, ridge_reg.predict(x_train_selection) )
print('MSE Train Data: ', mse)
mse = np.sqrt(mean_squared_error(y_train, ridge_reg.predict(x_train_selection) ))
print('RMSE Train Data: ', mse)
mae = mean_absolute_error(y_train, ridge_reg.predict(x_train_selection) )
print('MAE Train Data: ', mae)

mse = mean_squared_error(y_test, ridge_reg.predict(x_test_selection) )
print('MSE Test Data: ', mse)
mse = np.sqrt(mean_squared_error(y_test, ridge_reg.predict(x_test_selection) ))
print('RMSE Test Data: ', mse)
mae = mean_absolute_error(y_test, ridge_reg.predict(x_test_selection) )
print('MAE Test Data: ', mae)

MSE Train Data:  0.5571018439143713
RMSE Train Data:  0.7463925534960617
MAE Train Data:  0.5341869158446886
MSE Test Data:  0.6149368722735827
RMSE Test Data:  0.7841791072666899
MAE Test Data:  0.5396717419461016


In [21]:
#finding a good alpha for Ridge regression

alphas = []
mses = []
min_mse = float('inf')
min_mse_alpha = 0
for alpha in range(1, 200, 1):
  ridge_reg_find_alph = Ridge(alpha=alpha).fit(x_train_selection, y_train)
  mse = mean_squared_error(y_test, ridge_reg_find_alph.predict(x_test_selection))
  
  if mse < min_mse:
    min_mse = mse
    min_mse_alpha = alpha
  
#   print(alpha, mse)
  
  alphas.append(alpha)
  mses.append(mse)
print(min_mse_alpha, min_mse)

1 0.6149368722735827
